In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [12]:
def calculate_angle(a,b,c):
    # a is first angle, b is second angle, c is 3rd angle
    a = np.array(a)
    b = np.array(b)
    c = np.array(b)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0/np.pi)
    #if (angle > 180):
       # return 360-angle;
    return angle

In [13]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1])) # confidence metric c, dont transform that
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) #.circle creates circle, 4 is size, 255 is color green, -1 is fill circle

In [15]:
# nose 0, left eye 1, right eye 2, left ear 3, right ear 4, 
# left shoulder 5, right shoulder 6, left elbow 7, right elbow 8, 
# left wrist 9, right wrist 10, left hip 11, right hip 12, left knee 13, right knee 14, left ankle 15, right ankle 16
# elbow, shoulder, hip, ankle

In [5]:
EDGES = {
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1-5]
        y2, x2, c2 = shaped[p2-5]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [16]:
frame = cv2.imread("squats-claudia-hold.png")
img = frame.copy()

img = tf.image.resize_with_pad(tf.expand_dims(img, axis = 0), 192, 192)
input_image = tf.cast(img, dtype=tf.float32)
        
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
interpreter.invoke()
kyps_w_scores = interpreter.get_tensor(output_details[0]['index'])
        # end of taking coordinates

        # 5,6 shoulder; 7,8 elbow; 11,12 hip; 13,14 knee; 15,16 ankle
        # 0,1           2,3         6,7        8,9        10,11
        # sectioning off all scores were interested in
"""wanted_scores = kyps_w_scores[0][0][5:]
        # all angles for hands - 1st left shoulder, hip, knee
back_angle = [calculate_angle(wanted_scores[0], wanted_scores[6], wanted_scores[8])] + [calculate_angle(wanted_scores[1], wanted_scores[7], wanted_scores[9])]
        # all angles for legs - 1st left hip, knee, ankle
knee_angle = [calculate_angle(wanted_scores[6], wanted_scores[8], wanted_scores[10])] + [calculate_angle(wanted_scores[1], wanted_scores[7], wanted_scores[11])]
"""

wanted_scores = kyps_w_scores[0][0][0:]
        # all angles for hands - 1st left shoulder, hip, knee
back_angle = [calculate_angle(wanted_scores[5], wanted_scores[11], wanted_scores[15])] + [calculate_angle(wanted_scores[6], wanted_scores[12], wanted_scores[16])]
        # all angles for legs - 1st left hip, knee, ankle
knee_angle = [calculate_angle(wanted_scores[11], wanted_scores[13], wanted_scores[15])] + [calculate_angle(wanted_scores[12], wanted_scores[14], wanted_scores[16])]


print(back_angle)
print(knee_angle)

# draw_keypoints(frame, wanted_scores, 0.4)
# draw_connections(frame, wanted_scores, EDGES, 0.4)

# cv2.imwrite("image_w_resized.png", frame)
        
cv2.destroyAllWindows()

[147.00199509079073, 140.98025443169738]
[113.897919165004, 114.67429993777019]


In [ ]:
[147.00199509079073, 140.98025443169738]
[113.897919165004, 140.98025443169738]

In [ ]:
[147.00199509079073, 140.98025443169738]
[113.897919165004, 114.67429993777019]